In [1]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig

/Users/arthurriche/opt/miniconda3/envs/LLM/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Charger le dataset avec une révision spécifique
dataset = load_dataset("knkarthick/dialogsum")
print(dataset)

# Visualiser les informations du dataset
print(dataset)

# Afficher un exemple du jeu de test
example_indices = [40, 200]
dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})
DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})
---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine b

In [3]:
model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)


In [4]:
sentence = "What time is it, Tom?"

# Encoder la phrase
sentence_encoded = tokenizer(sentence, return_tensors='pt')

# Décoder la phrase encodée
sentence_decoded = tokenizer.decode(
    sentence_encoded["input_ids"][0],
    skip_special_tokens=True
)

# Afficher les résultats
print('ENCODED SENTENCE:')
print(sentence_encoded["input_ids"][0])
print('\nDECODED SENTENCE:')
print(sentence_decoded)

ENCODED SENTENCE:
tensor([ 363,   97,   19,   34,    6, 3059,   58,    1])

DECODED SENTENCE:
What time is it, Tom?


In [5]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    inputs = tokenizer(dialogue, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs=inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print(f'Example {i + 1}')
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - WITHOUT PROMPT ENGINEERING:
Person1: It's ten to nine.

--------------------------------

In [6]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
    Summarize the following conversation.

    {dialogue}

    Summary:
    """

    # Input constructed prompt instead of the dialogue
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs=inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print(f'Example {i + 1}')
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

    Summarize the following conversation.

    #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

    Summary:
    
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
The train 

In [7]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ""
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']
        
        # La séquence de fin '{summary}\n\n' est importante pour FLAN-T5. D'autres modèles peuvent différer.
        prompt += f"""
        Dialogue:
        {dialogue}

        What was going on?
        {summary}
        """

    dialogue = dataset['test'][example_index_to_summarize]['dialogue']
    prompt += f"""
    Dialogue:
    {dialogue}

    What was going on?
    """
    
    return prompt


In [8]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)



        Dialogue:
        #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

        What was going on?
        #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
        
    Dialogue:
    #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a fast

In [9]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs=inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


In [10]:
example_indices_full = [40, 80, 120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


        Dialogue:
        #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

        What was going on?
        #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
        
        Dialogue:
        #Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable p

In [11]:
example_indices_full = [40, 80, 120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)

summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs=inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')

Token indices sequence length is longer than the specified maximum sequence length for this model (819 > 512). Running this sequence through the model will result in indexing errors



        Dialogue:
        #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

        What was going on?
        #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
        
        Dialogue:
        #Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable p

Inference Parameters

In [12]:
#generation_config = GenerationConfig(max_new_tokens=50)
# generation_config = GenerationConfig(max_new_tokens=10)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.1)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.5)
generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=1.0)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs=inputs["input_ids"],
        generation_config=generation_config,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
People would like to upgrade their computers and operating system in their new software.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.



In [13]:
'''Lab 2: Fine-tuning a model on the dialogsum dataset'''
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [21]:
model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [22]:
def print_number_of_trainable_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
        
    return f'trainable parameters: {trainable_model_params},\n all model parameters: {all_model_params} \n percentage of trainable parameters: {trainable_model_params/all_model_params * 100}%'

print(print_number_of_trainable_parameters(model))

trainable parameters: 247577856,
 all model parameters: 247577856 
 percentage of trainable parameters: 100.0%


In [23]:
# Full fine-tuning
import numpy as np
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example["input_ids"] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example["labels"] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary'])

Map: 100%|██████████| 500/500 [00:00<00:00, 1605.71 examples/s]


In [24]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index%100==0, with_indices=True)

Filter: 100%|██████████| 500/500 [00:00<00:00, 2396.87 examples/s]


In [25]:
print("Shapes of the datasets:")
print(f"Training: {len(tokenized_datasets['train'])} examples")
print(f"Validation: {len(tokenized_datasets['validation'])} examples")
print(f"Test: {len(tokenized_datasets['test'])} examples")

print(tokenized_datasets)

Shapes of the datasets:
Training: 125 examples
Validation: 5 examples
Test: 15 examples
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 125
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
})


In [26]:
output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

In [ ]:
trainer.train()

In [ ]:
instruct_model = model
# Puis on l'évalue
index = 200

dialogue = dataset['test'][index]['dialogue']
human_baseline_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("mps")  # Déplacer les entrées sur MPS
model.to("mps")

model_outputs = model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
model_text_output = tokenizer.decode(model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')

/Users/arthurriche/opt/miniconda3/envs/LLM/lib/python3.9/site-packages/transformers/pytorch_utils.py:332: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
#Person1#: Have you considered upgrading your system? #Person2#: Yes, but I'm not sure what exactly I would need. #Person2#: You could add ep a painting program to your software. #Person2#: You'd probably need a faster processor, more memory and a faster modem. Do e n you have a CD-ROM drive? #Person2#: No, yep.
---------------------------------------------------------------------------------------------------
INSTRUCT MODEL:
#Person2#: You can add a DVD-ROM drive.


In [38]:
rouge = evaluate.load('rouge')
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

# Tokenization en batch
tokenizer.pad_token = tokenizer.eos_token  # Gérer les padding si nécessaire
input_ids = tokenizer(dialogues, return_tensors="pt", padding=True, truncation=True).input_ids
input_ids = input_ids.to("mps")  # Déplacer les entrées sur MPS

# Génération des résumés par les deux modèles
original_model_outputs = model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
original_model_summaries = [tokenizer.decode(output, skip_special_tokens=True) for output in original_model_outputs]

instruct_model_outputs = model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
instruct_model_summaries = [tokenizer.decode(output, skip_special_tokens=True) for output in instruct_model_outputs]

# Création du DataFrame pour comparer les résumés
df = pd.DataFrame(
    list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries)),
    columns=['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries']
)

# Évaluation avec la métrique ROUGE
rouge = evaluate.load('rouge')
rouge_scores = rouge.compute(predictions=original_model_summaries, references=human_baseline_summaries)

# Affichage des scores ROUGE
print("ROUGE Scores:", rouge_scores)

/Users/arthurriche/opt/miniconda3/envs/LLM/lib/python3.9/site-packages/transformers/pytorch_utils.py:332: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)


ROUGE Scores: {'rouge1': 0.08275620213120213, 'rouge2': 0.004, 'rougeL': 0.06734681984681984, 'rougeLsum': 0.06516317016317016}


In [39]:
# PEFT Method !

from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32,  # Rank de la factorisation (contrôle la compression)
    lora_alpha=32,  # Facteur de mise à l’échelle pour les poids LoRA
    target_modules=["q", "v"],  # Ciblage des modules de l'attention (query et value)
    lora_dropout=0.05,  # Probabilité de dropout pour éviter l'overfitting
    bias="none",  # Pas de bias supplémentaire
    task_type=TaskType.SEQ_2_SEQ_LM  # Type de tâche (Seq2Seq pour Flan-T5)
)
peft_model = get_peft_model(model, lora_config)

print_number_of_trainable_parameters(peft_model)


'trainable parameters: 3538944,\n all model parameters: 251116800 \n percentage of trainable parameters: 1.4092820552029972%'

In [40]:
output_dir = f'./peft-dialogue-summary-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)


In [ ]:
from peft import PeftModel, PeftConfig
peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

peft_model = PeftModel.from_pretrained(
    peft_model_base,
    './peft-dialogue-summary-checkpoint-from-s3/',  # Chemin du checkpoint
    torch_dtype=torch.bfloat16,
    is_trainable=False
)


In [26]:
API_KEY = "g3hpThh9vpGP5m6tko1gEWOfeL6kmpJu"
ENDPOINT_FINE_TUNE = "https://api.mistral.ai/mistral-large-latest"
import requests

In [27]:
prompt = "Début : Le soleil se levait lentement à l'horizon. ### Fin : Les oiseaux chantaient et la journée promettait d'être magnifique. ### Remplis le milieu :"

data = {
    "model": "mistral-large-latest",
    "prompt": prompt,
    "max_tokens": 50,
    "temperature": 0.7
}

response = requests.post(ENDPOINT_FINE_TUNE, headers=headers, json=data)
print(response.json()["choices"][0]["text"])

KeyError: 'choices'

In [18]:
import requests

# Configuration
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

data = {
    "model": "mistral-7b",  # Modèle de base à fine-tuner
    "training_file": "https://path_to_your_data_file.jsonl",  # URL de votre fichier JSONL
    "validation_file": "https://path_to_your_validation_file.jsonl",  # (optionnel) Fichier de validation
    "hyperparameters": {
        "epochs": 3,  # Nombre de passages sur les données
        "batch_size": 16,  # Taille des lots
        "learning_rate": 5e-5  # Taux d'apprentissage
    }
}

# Envoyer la requête
response = requests.post(ENDPOINT_FINE_TUNE, headers=headers, json=data)

# Vérifier la réponse
if response.status_code == 200:
    print("Fine-tuning lancé :", response.json())
else:
    print("Erreur :", response.text)

Erreur : {
  "message":"no Route matched with those values",
  "request_id":"d3b22012c9091f0b0b39704b1a906c7f"
}


In [19]:
job_id = "fine_tune_job_id"

status_endpoint = f"https://api.mistral.ai/v1/fine-tune/{job_id}/status"

response = requests.get(status_endpoint, headers=headers)

if response.status_code == 200:
    print("Statut du fine-tuning :", response.json())
else:
    print("Erreur :", response.text)

Erreur : {
  "message":"no Route matched with those values",
  "request_id":"0f7b9967107dcf237787e314053e8529"
}


In [20]:
inference_data = {
    "model": "fine-tuned-model-id",  # ID du modèle fine-tuné
    "prompt": "Explique les avantages du produit X.",
    "max_tokens": 100
}

response = requests.post("https://api.mistral.ai/v1/completions", headers=headers, json=inference_data)

if response.status_code == 200:
    print("Réponse générée :", response.json()["choices"][0]["text"])
else:
    print("Erreur :", response.text)

Erreur : {"detail":[{"type":"missing","loc":["body","messages"],"msg":"Field required","input":{"model":"fine-tuned-model-id","prompt":"Explique les avantages du produit X.","max_tokens":100}}]}
